# Feature Engineering: ICU Traversal

In [1]:
import psycopg2
from psycopg2 import extras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools

%matplotlib inline

In [20]:
def cursor_connect(cursor_factory=None):
    """
    Connects to the DB and returns the connection and cursor, ready to use.
    
    Parameters
    ----------
    cursor_factory : psycopg2.extras
    
    Returns
    -------
    (psycopg2.extensions.connection, psycopg2.extensions.cursor)
        A tuple of (psycopg2 connection, psycopg2 cursor).
    """
    #DB connection
    conn = psycopg2.connect(dbname="mimic", user="mimic", host="localhost", port="2345",
                            password="oNuemmLeix9Yex7W")
    if not cursor_factory:
        cur = conn.cursor()
    else:
        cur = conn.cursor(cursor_factory=cursor_factory)
    return conn, cur

def exec_query(query, curs_dict=True):
    """
    Execute query and returns the SQL output.
    
    Parameters
    ----------
    query: string containing SQL SELECT command
    curs_dict: dict cursor factory (output is dict)
    
    Returns
    -------
    rows: list of rows/records (SQL output)
    """
    if curs_dict == True: 
        conn, cur = cursor_connect(psycopg2.extras.DictCursor)
    else:
        conn, cur = cursor_connect()
    cur.execute(query)
    rows = cur.fetchall()
    return rows

## N<sub>readmissions</sub>
Feature: The number of ICU readmissions a patient has had. The value is not tied to a hospital admission and is the same for the same `subjectid`, despite multiple records/rows.

In [24]:
# query
q_icupat="""SELECT * FROM
    (SELECT subject_id, COUNT(icustay_id) AS n_icustays
    FROM icustays
    GROUP BY subject_id) AS sub_q
WHERE n_icustays > 1;"""

# Query output
icupat = exec_query(q_icupat, False)
df_icupat = pd.DataFrame(icu_stay, columns=['subjectid', 'n_icustays'])
df_icupat

n_readm = pd.Series(df_icupat.n_icustays - 1, name='n_readm')
df_icu = pd.concat([df_icupat.subjectid, n_readm], axis=1)
df_icu

,subjectid,n_readm
0,7,1
1,17,1
2,21,1
3,23,1
4,34,1
5,36,2
6,41,1
7,61,1
8,67,1
9,68,2


## N<sub>trav</sub>
Feature: The number of total ward transfers for each patient's hospital admission.

In [27]:
q_mult= """SELECT subject_id, hadm_id, icustay_id, eventtype,
prev_careunit, curr_careunit, prev_wardid, curr_wardid, intime, outtime, los
FROM transfers;"""
mult_trav = exec_query(q_mult, False)
mult_col = ['subjectid', 'hadmid', 'icustayid', 'eventtype', 'prev_cu', 'curr_cu', 
       'prev_wardid', 'curr_wardid', 'intime', 'outtime', 'los']
df_mult = pd.DataFrame(mult_trav, columns = mult_col)
df_mult.replace(to_replace='', value=np.nan, inplace=True, regex=True)

# filter for ICU patients with readmissions
filter_preadm = list(df_icupat.subjectid)
df_mult_readm = df_mult[df_mult.subjectid.isin(filter_preadm)]

# filter for exclusion of neonate patients
df_mult_readm = df_mult_readm[df_mult_readm['prev_cu'] != 'NWARD']
df_mult_readm = df_mult_readm[df_mult_readm['prev_cu'] != 'NICU']
df_mult_readm = df_mult_readm[df_mult_readm['curr_cu'] != 'NWARD']
df_mult_readm = df_mult_readm[df_mult_readm['curr_cu'] != 'NICU']

In [28]:
df_mult_readm_grp = df_mult_readm.groupby(['subjectid', 'hadmid']).size()
df_mult_readm_grp = df_mult_readm_grp.to_frame(name='n_trav').reset_index()

# join DF on subjectid to add n_icustays col
df_icu1 = pd.merge(df_mult_readm_grp, df_icu, on='subjectid', 
                        how='left')
df_icu1

,subjectid,hadmid,n_trav,n_readm
0,17,161087,4,1
1,17,194023,4,1
2,21,109451,6,1
3,21,111970,2,1
4,23,124321,5,1
5,23,152223,4,1
6,34,115799,4,1
7,34,144319,5,1
8,36,122659,6,2
9,36,165660,5,2


## N<sub>icutrav</sub>
Feature: The number of total *ICU* ward transfers for each patient's hospital admission. Only transfers into the ICU, which is assigned a `icustay_id` is accounted for.

In [29]:
df_mult_readm_icu = df_mult_readm[df_mult_readm.icustayid.notnull()==True]
df_mult_readm_hadm = df_mult_readm_icu.groupby(['subjectid', 'hadmid']).size().to_frame('n_icutrav').reset_index()

# join DF  on subjectid to add n_readm col
df_icu2 = pd.merge(df_icu1, df_mult_readm_hadm.loc[:, ['hadmid', 'n_icutrav']], 
                   on='hadmid', how='inner')
df_icu2

,subjectid,hadmid,n_trav,n_readm,n_icutrav
0,17,161087,4,1,1
1,17,194023,4,1,1
2,21,109451,6,1,2
3,21,111970,2,1,1
4,23,124321,5,1,1
5,23,152223,4,1,1
6,34,115799,4,1,1
7,34,144319,5,1,2
8,36,122659,6,2,3
9,36,165660,5,2,1


## N<sub>ICU Stays</sub>
Feature: the number of unique ICU stays for a patient's hospital admission.

In [34]:
# query
q_icustay = """SELECT subject_id, hadm_id, COUNT(DISTINCT icustay_id)
FROM transfers
GROUP BY subject_id, hadm_id;
"""

# Query output
icustay = exec_query(q_icustay, False)
df_icustay = pd.DataFrame(icustay, columns=['subjectid', 'hadmid', 
                                            'n_icustays'])
df_icustay

# join DF  on subjectid to add n_readm col
df_icu3 = pd.merge(df_icu2, df_icustay.loc[:, ['hadmid', 'n_icustays']], 
                   on='hadmid', how='inner')
df_icu3

,subjectid,hadmid,n_trav,n_readm,n_icutrav,n_icustays
0,17,161087,4,1,1,1
1,17,194023,4,1,1,1
2,21,109451,6,1,2,1
3,21,111970,2,1,1,1
4,23,124321,5,1,1,1
5,23,152223,4,1,1,1
6,34,115799,4,1,1,1
7,34,144319,5,1,2,1
8,36,122659,6,2,3,1
9,36,165660,5,2,1,1


## Traversal Pairs

In [43]:
df_trav_copy = df_mult_readm_icu.copy()
df_trav_copy.prev_cu.fillna('nonicu', inplace=True)
df_trav_copy.curr_cu.fillna('nonicu', inplace=True)
df_trav_copy['trans'] = df_trav_copy.prev_cu+'-'+df_trav_copy.curr_cu

df_toppairs = df_trav_copy.trans.value_counts(ascending=False).to_frame()
df_top = df_toppairs.transpose().iloc[:, 0:11] # transpose to columns

In [36]:
list(df_top.columns)

['nonicu-MICU',
 'nonicu-SICU',
 'nonicu-CCU',
 'nonicu-CSRU',
 'nonicu-TSICU',
 'MICU-MICU',
 'CSRU-CSRU',
 'SICU-SICU',
 'TSICU-TSICU',
 'CCU-CCU',
 'CCU-MICU']

In [64]:
from collections import Counter
# a = dict(Counter(df_trav_copy[df_trav_copy.subjectid==34].trans))
# a['subjectid']=34
# a

sid = list(df_trav_copy.subjectid.value_counts().index) # unique subject_id

main_d = dict()
for subj in sid:
    pair_d = dict(Counter(df_trav_copy[df_trav_copy.subjectid==subj].trans))
    pair_d['subjectid'] = subj # add subjectid key
    main_d[subj] = pair_d

In [66]:
df_pairct = pd.DataFrame.from_dict(main_d, orient='index')

# drop non-top trans pair cols
pairs_drop = list(df_toppairs.iloc[10:].index)
df_pairct.drop(pairs_drop, axis=1, inplace=True) 

df_icu4 = pd.merge(df_icu3, df_pairct, on='subjectid', how='left')
df_icu4

,subjectid,hadmid,n_trav,n_readm,n_icutrav,n_icustays,nonicu-MICU,nonicu-SICU,nonicu-TSICU,nonicu-CSRU,MICU-MICU,TSICU-TSICU,nonicu-CCU,CCU-CCU,CSRU-CSRU,SICU-SICU
0,17,161087,4,1,1,1,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
1,17,194023,4,1,1,1,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
2,21,109451,6,1,2,1,1.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN
3,21,111970,2,1,1,1,1.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN
4,23,124321,5,1,1,1,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
5,23,152223,4,1,1,1,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
6,34,115799,4,1,1,1,1.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN
7,34,144319,5,1,2,1,1.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN
8,36,122659,6,2,3,1,1.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,2.0,NaN
9,36,165660,5,2,1,1,1.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,2.0,NaN


## Average ICU LOS
Feature (Response): The average length of stay in the ICU (`icustay_id`) for each patient's hospital admission.
  * Feature is the response/dependent variable (`y`)

**Note**: The overall hospital admission LOS may be extracted by taking the aggregate mean of the DataFrame, after it has been grouped by `subject_id`.

In [40]:
# Average ICU LOS
avgiculos = df_mult_readm_icu.groupby(['subjectid', 'hadmid'])['los'].mean()
df_avgiculos = avgiculos.to_frame(name='avg_iculos').reset_index()

# Merge
df_icu5 = pd.merge(df_icu4, df_avgiculos.loc[:,['hadmid', 'avg_iculos']], on='hadmid', how='left')
# df_icu2.groupby(['subjectid'])['avg_iculos'].mean() # overall LOS
df_icu5

,subjectid,hadmid,n_trav,n_readm,n_icutrav,n_icustays,nonicu-MICU,nonicu-SICU,nonicu-TSICU,nonicu-CSRU,MICU-MICU,TSICU-TSICU,nonicu-CCU,CCU-CCU,CSRU-CSRU,SICU-SICU,avg_iculos
0,17,161087,4,1,1,1,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,21.110000
1,17,194023,4,1,1,1,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,49.700000
2,21,109451,6,1,2,1,1.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,70.825000
3,21,111970,2,1,1,1,1.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,200.750000
4,23,124321,5,1,1,1,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,28.470000
5,23,152223,4,1,1,1,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,30.340000
6,34,115799,4,1,1,1,1.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,17.280000
7,34,144319,5,1,2,1,1.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,18.690000
8,36,122659,6,2,3,1,1.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,2.0,NaN,54.873333
9,36,165660,5,2,1,1,1.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,2.0,NaN,104.360000
